<a href="https://colab.research.google.com/github/PrinceChaudhary1962/Movie-Reservation-System/blob/main/Movie_reservation_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
!pip install fastapi uvicorn nest_asyncio pyngrok


In [15]:
import nest_asyncio
from pyngrok import ngrok

# Allow nested loops for Colab
nest_asyncio.apply()

# Kill any previous ngrok tunnels
ngrok.kill()


In [16]:
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
from typing import List

app = FastAPI(title="Movie Reservation System")

# Mock database
movies = {
    1: {"title": "Inception", "seats": 5, "reservations": []},
    2: {"title": "Interstellar", "seats": 3, "reservations": []},
    3: {"title": "The Dark Knight", "seats": 4, "reservations": []},
}

class Reservation(BaseModel):
    user: str
    movie_id: int

@app.get("/movies")
def get_movies():
    """Return movies as a list instead of dict (better for frontend)"""
    return [{"id": mid, **details} for mid, details in movies.items()]

@app.post("/reserve")
def reserve_seat(reservation: Reservation):
    if reservation.movie_id not in movies:
        raise HTTPException(status_code=404, detail="Movie not found")

    movie = movies[reservation.movie_id]

    if movie["seats"] <= 0:
        raise HTTPException(status_code=400, detail="No seats available")

    movie["reservations"].append(reservation.user)
    movie["seats"] -= 1
    return {"message": f"Seat reserved for {reservation.user} in {movie['title']}"}

@app.get("/reservations/{movie_id}")
def get_reservations(movie_id: int):
    if movie_id not in movies:
        raise HTTPException(status_code=404, detail="Movie not found")

    return {
        "movie": movies[movie_id]["title"],
        "reservations": movies[movie_id]["reservations"],
    }


In [18]:
import uvicorn
import threading

def run():
    uvicorn.run(app, host="0.0.0.0", port=8000, log_level="info")

# Run FastAPI in background only once
thread = threading.Thread(target=run, daemon=True)
thread.start()

# Create new ngrok tunnel
tunnel = ngrok.connect(8000)
print("Public URL:", tunnel.public_url)
print("Docs:", tunnel.public_url + "/docs")


INFO:     Started server process [24211]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
ERROR:asyncio:Task exception was never retrieved
future: <Task finished name='Task-80' coro=<Server.serve() done, defined at /usr/local/lib/python3.12/dist-packages/uvicorn/server.py:69> exception=SystemExit(1)>
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/uvicorn/server.py", line 164, in startup
    server = await loop.create_server(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.12/asyncio/base_events.py", line 1584, in create_server
    raise OSError(err.errno, msg) from None
OSError: [Errno 98] error while attempting to bind on address ('0.0.0.0', 8000): [errno 98] address already in use

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/lib/python3.12/threading.py", line 1075, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.12/

Public URL: https://f889afb23a5b.ngrok-free.app
Docs: https://f889afb23a5b.ngrok-free.app/docs
